# install modules

In [ ]:
pip install langchain-community
pip install openAI

In [29]:
import pandas as pd # allows us to work with tabular data
import requests # to send the API requests to Elsevier
import json # to read the JSON data that is returned by the APIs
import pickle # pickle files are a good way to save data for reuse in Python
from datetime import datetime
from langchain_community.chat_models import ChatOpenAI
from IPython.display import display, Image
from openai import OpenAI

# user  input 
- 부여받은 SCOPUS_API_KEY와  OPEN_API_KEY를 입력 후 찾고자하는 주제, 탐색 년도를 입력한다.

In [20]:
api_key = ''
OPEN_API_KEY = ''

search_keyword = "5G"
start_year = 2015
end_year = 2023


## SCOPUS_API 통신을 위한 함수 구현

In [21]:


search_string_parameters = {
    'ISSN' : '', # Limit results to results from a journal ISSN (add as a string). Leave = '' if no value.
    'AUTHOR-NAME' : '', # Search the author name field. lastname, firstname. e.g., Noble, Safiya
    'KEY' : '', # Add a keyword to search in the text of the article
    'PUBLISHER' : '', # Add a publisher name. e.g., Springer
    'EXACTSRCTITLE' : 'journal', # Add keywords that appear in the journal, book, or conference title. e.g., Informatics
    'AFFIL' : '', # add keywords that appear in the author's institutional affiliation. e.g., to find the University of Minnesota, search for University and Minnesota
    'TITLE' : search_keyword, # Add keywords that appear in the article or chapter title. Can use AND, OR, and AND NOT. e.g. cat AND dog 
    'OPENACCESS' : '1', # Add 1 to limit your results to open access articles or 0 for articles that are not open access
    'start_year' : start_year, # Limit your search to items published after this year. Leave = 0 if no value. YYYY , e.g., 1995
    'end_year' : end_year # Limit your search to items published before this year. Leave = 0 if no value. YYYY, e.g., 2020
}
def search_builder(search_string_parameters):
    ''' Concatenates a search string query formatted for the Scopus Search API.
    - search_string_parameters expects a python dictionary with keys aligned to API search fields.
    '''
    search_string = ''

    for k,v in search_string_parameters.items():
        if k == 'start_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR > {v}'
        elif k == 'end_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR < {v}'
        elif v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'{k}({v})'

    return search_string

def create_url(search_string):
    '''Accepts a formatted search string that will be added to the Scopus Search API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Scopus Search API.'''
    
    query = f'{search_string}'
    url_template = 'https://api.elsevier.com/content/search/scopus?query={query}&apiKey={api_key}'
    full_url = url_template.format(query=query, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url, params={'cursor': '*'}, next_ = '*'):
    '''Accepts API URL with ISSN, default parameters, and next page cursor;
    Sends request to Scopus API and collects JSON results for each call;
    Returns r.json() for the ['search-results'] key.'''
    
    params['cursor'] = next_
    r = requests.get(full_url, params=params)
    r.raise_for_status()
    return r.json()['search-results'], r.headers

In [22]:
search_string = search_builder(search_string_parameters)
print(search_string)
full_url = create_url(search_string)
r_json, r_headers = connect_to_endpoint(full_url)

EXACTSRCTITLE(journal) AND TITLE(5G) AND OPENACCESS(1) AND PUBYEAR > 2015 AND PUBYEAR < 2023


## API response에 대한 json format

In [23]:
print('Title:', r_json['entry'][0]['dc:title'], 
      '\nCreator:', r_json['entry'][0]['dc:creator'], 
      '\nPublication:', r_json['entry'][0]['prism:publicationName'],
      '\nDate:', r_json['entry'][0]['prism:coverDate'],
      '\nISSN:', r_json['entry'][0]['prism:url'], 
     '\nAbstract', r_json['entry'][0])

Title: Integrated Microwave and mm-Wave MIMO Antenna Module With 360° Pattern Diversity for 5G Internet of Things 
Creator: Hussain N. 
Publication: IEEE Internet of Things Journal 
Date: 2022-12-15 
ISSN: https://api.elsevier.com/content/abstract/scopus_id/85135752407 
DOI: 10.1109/JIOT.2022.3194676 
Abstract {'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85135752407'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85135752407?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85135752407&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85135752407&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85135752407', 'dc:identifier': 'SCOPUS_ID:851357524

In [24]:
dois = [entry.get('prism:doi') for entry in r_json.get('entry', [])]
print(dois[0])
print(len(dois))

10.1109/JIOT.2022.3194676


## 한건의 요청당 최대 25개의 논문이 검색된다.

In [26]:

dois = dois[:3]
print(len(dois))


3
3


## doi 추출 후 doi를 이용해 abstract를 가져와보자 

In [27]:
def create_url(doi):
    """Accepts a DOI as a string that will be added to the Abstract Retrieval API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Abstract Retrieval API.
    """
    doi = f'{doi}'
    url_template = 'https://api.elsevier.com/content/abstract/doi/{doi}?&apiKey={api_key}'
    full_url = url_template.format(doi=doi, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url):
    '''Input full_url from create_url function;
    Send request to Scopus Abstract Retrieval API
    Returns r.json response;'''
    
    r = requests.get(full_url, headers =  {'Accept': 'application/json'})
    r.raise_for_status()
    return r.json(), r.headers

## OPEN_AI 연동

In [30]:
chat_model = ChatOpenAI(openai_api_key=OPEN_API_KEY)
client = OpenAI(api_key=OPEN_API_KEY)

## ChatGPT, DALL-E 에 명령어 전달
- 논문제목, 출판지, 인용횟수, ChatGPT 를 통해 요약된 abstract, 요약된 abstract 기반으로 생성된 image

In [57]:
for index, doi in enumerate(dois):
    full_url = create_url(doi)
    r_json, r_headers = connect_to_endpoint(full_url)
    try:
        r_json = r_json['abstracts-retrieval-response']
        abstract = r_json['coredata']['dc:description']
        print(index)
        title = r_json['coredata']['dc:title']
        publication_name = r_json['coredata']['prism:publicationName']
        citedby_count = r_json['coredata']['citedby-count']          
        url = r_json['coredata']['prism:url']
        print('논문 제목  :  ' + title)
        print('출판지  :  ' + publication_name)
        print('인용횟수  :  ' + citedby_count)
        print(url)
        result = chat_model.predict(abstract + " 이 내용을 100자 내외로 요약해줘")
        print(result)
        response = client.images.generate(
                        model="dall-e-3",
                        prompt=result,
                        size="1024x1024",
                        quality="standard",
                        n=1,
                    )

        image_url = response.data[0].url
        display(Image(url=image_url))
        print(r_json['coredata']['dc:description'])
        
    except KeyError:
        ## abstract이 수록되지 않은 논문도 있음 
        print('예외가 발생한 부분')
        print(r_json['coredata']['dc:title'])
        print(index)

0
논문 제목  :  Integrated Microwave and mm-Wave MIMO Antenna Module With 360° Pattern Diversity for 5G Internet of Things
출판지  :  IEEE Internet of Things Journal
인용횟수  :  27
https://api.elsevier.com/content/abstract/scopus_id/85135752407
본 연구에서는 5G IoT 시스템을 위해 마이크로파(2.5/3.5/5.5/7.5 GHz) 및 mm파 대역(23-31 GHz)에서 작동하는 멀티밴드 안테나와 그의 12포트 MIMO 구성을 제안한다. 안테나는 잘 설계된 1/4 파장 스텁을 추가하여 멀티밴드 특성을 얻는다. 또한, 안테나는 2.37-2.65, 3.25-3.85, 5.0-6.1, 7.15-8.5 GHz 주파수 대역에서 중요한 성능을 발휘하며, mm파 대역에서는 23-31 GHz에서 원하는 방사 특성을 갖는다. 또한, 안테나는 95% 이상의 방사 효율과 안정된 이득 특성을 갖추고 있다. 이 단일 안테나는 2× 2 MIMO 안테나를 설계하기 위해 활용되며, 이 MIMO 단위는 2× 4 및 제안된 3× 4 (12포트) MIMO 구성에서 스펙트럼과 패턴 다양성을 달성하기 위해 추가로 활용된다. 안테나 요소의 독특한 3차원 배열을 고려할 때, 12포트 MIMO 시스템은 마이크로파와 mm파 안테나의 공동 설계, 좋은 격리 및 패턴 다양성을 제공하여 고도 및 방위 평면에서 완전한 360° 공간 커버리지를 제공하는 유일한 시스템이다. 제안된 안테나 모듈은 스마트 하우스, 사무실 및 차량간 통신과 같은 실내 시나리오에서 특히 5G IoT에 적합하다.


The complete coverage of the operating frequency bands from microwave bands to millimeter (mm-wave) is required for the realization of the fifth-generation (5G) Internet of Thing (IoT) systems. Here, we present a multiband antenna operating at the microwave (2.5/3.5/5.5/7.5 GHz) and mm-wave bands (23-31 GHz), and its 12-port MIMO configuration with pattern diversity affording 360° coverage for 5G IoT applications. The multiband characteristics are obtained by adding well-designed quarter-wavelength stubs. The antenna operates at the important frequency bands from 2.37-2.65, 3.25-3.85, 5.0-6.1, and 7.15-8.5 GHz (|S11| <-10 dB), while it resonates from 23-31 GHz at the mm-wave band with the desired radiation characteristics. Moreover, the antenna has more than 95% radiation efficiency and a stable gain (> 2.5 dBi at microwave band and 6.5 dBi at mm-wave bands) characteristics. In addition, the single-element antenna is translated to design a 2× 2 MIMO antenna. This MIMO unit is further u

It is challenging to design a compact antenna pair with isolation better than 20 db and with robustness to antenna position. In this paper, a T-shaped dummy antenna is adopted to decouple the tightly arranged antenna pair for 5G mobile devices. Working in the 3.8 GHz of N77 band, the proposed antenna pair is with an impedance bandwidth of 280 MHz. Isolation of the proposed antenna pair is enhanced from 13 db to 30 db by adding the T-shaped dummy antenna, and it is with high robustness even though the antenna position is altered. The calculation process of the loading requirement of T-shaped dummy antenna is shown. By duplicating the proposed antenna pair, a 4×4 MIMO antenna with high isolation is proposed. The measured results show the MIMO antenna with isolation better than 31 db, and the average total efficiency is about 49%, and the Envelope Correlation Coefficient (ECC) is lower than 0.05 in the -6 db pass band. To the best of the authors' knowledge, compared to most of the reporte

– A broadband decoupled dual antenna pair suitable for sub-6G mobile terminal application is proposed and designed. The multiple input multiple output (MIMO) antenna pair is arranged on the metal frame of the mobile phone, with two antenna pairs on each metal frame. The slotted antenna element structure and defect ground decoupling structure are used to achieve wide band and high isolation. Based on the decoupled antenna pair, an eight-element MIMO antenna is designed. Similarly, the coupling between antenna pairs is effectively reduced by etching the decoupled defect structure of a zigzag shape on the system surface. The prototype of the antenna array is successfully fabricated and tested. Simulation and experimental results show that the proposed eight-element MIMO antenna can fully cover the n77/n78/n79 band of 5G New Radio and 5 GHz band of WLAN. The overall efficiency of the antenna is 50%-75%, the envelope correlation coefficient (ECC) is below 0.08, and the isolation between any